In [ ]:
# 以下のtrainデータから特長量を作成し、学習モデルを生成させる。
# /mnt/c/Users/hayat/Desktop/keiba_analysis/data_for_train/train/2005_2022

In [ ]:
!pip install lightGBM

In [ ]:
# /mnt/c/Users/hayat/Desktop/keiba_analysis/data_for_train/train/2005_2022
# のデータを取り込む
from pathlib import Path
import warnings
from sklearn.svm import SVC, LinearSVC
from sklearn import multiclass
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_sample_weight
import scipy.stats
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import tqdm
# 指定したディレクトリ配下のcsvデータをreadする
train_path = '/mnt/c/Users/hayat/Desktop/keiba_analysis/data_for_train/train/2005_2022'
def read_csv_data(train_path):
    # csvデータを読み込み、1行にまとめる
    p = Path(train_path)
    files = list(p.glob("*.csv"))
    df = pd.DataFrame()
    for i, file in tqdm.tqdm(enumerate(files)):
        race_csv_data = pd.read_csv(file, encoding='utf-8')
        # 各CSVファイルのデータを1行にまとめる
        race_csv_data = race_csv_data.sum(axis=0).to_frame().T
        
        # データフレームに追加
        df = pd.concat([df, race_csv_data], ignore_index=True)
        if i > 1:
            # データが多すぎるため、一旦100個で
            break
    return df
train_data = read_csv_data(train_path)

print("train data length {}".format(len(train_data)))

0it [00:00, ?it/s]


NameError: name 'race' is not defined

In [7]:
train_data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,パッションローズ,6,2,1,504,0,54.0,",2歳新馬",16,1200,",芝",藤岡佑介,4.7,1001,2歳新馬,",1回札幌7日目",1
1,1,マイネルカルナバル,9,2,0,474,0,54.0,",2歳新馬",16,1200,",芝",横山典弘,6.9,1001,2歳新馬,",1回札幌7日目",2
2,2,バクシンミノル,7,2,0,484,0,54.0,",2歳新馬",16,1200,",芝",古川吉洋,3.6,1001,2歳新馬,",1回札幌7日目",3
3,3,ニシノマルス,11,2,0,464,0,54.0,",2歳新馬",16,1200,",芝",岩田康誠,4.0,1001,2歳新馬,",1回札幌7日目",4
4,4,グラーフ,12,2,1,480,0,54.0,",2歳新馬",16,1200,",芝",北村浩平,22.0,1001,2歳新馬,",1回札幌7日目",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,10,モリトヴィーナス,3,6,1,462,2,55.0,",3歳以上1000万下",15,1800,",ダ",吉田隼人,266.8,1015,松戸特別,",4回中山3日目",11
11,11,ビービーファング,2,4,0,468,2,57.0,",3歳以上1000万下",15,1800,",ダ",小野次郎,50.7,1015,松戸特別,",4回中山3日目",12
12,12,マイネルハーバード,5,3,0,448,-6,54.0,",3歳以上1000万下",15,1800,",ダ",田中博康,112.4,1015,松戸特別,",4回中山3日目",13
13,13,マイネルトイフェル,6,5,0,476,-4,57.0,",3歳以上1000万下",15,1800,",ダ",長谷川浩,288.9,1015,松戸特別,",4回中山3日目",14
